In [ ]:
# Ломачук Ю.
import numpy as np
import scipy.special as sp

In [ ]:
_lmax = 20
_ps_max=100
_gaus_int = sp.gamma((np.arange(_lmax+1)+1)*5e-1)
_gaus_int[1::2]=0e0

# Вычисление двухэлектронных кулоновских интегралов
Рассмотрим вычисление интегралов вида 
$$
I(ABCD) = \int G(\vec r;\vec{R}_a, \alpha_a,\{p_a\}) G(\vec r; \vec{R}_b, \alpha_b,\{p_b\}) G(\vec r'; \vec{R}_c, \alpha_c, \{p_c\})G(\vec r'; \vec{R}_d, \alpha_d, \{p_d\})\frac{1}{|\vec{r}-\vec{r'}|}dV dV',
$$

интегрирование ведется по всему обьему, а ф-и G являются произведениями гауссиан на полиномы от $\vec{r}$:
$$
G(\vec{r}; \vec{R_a}, \alpha_a, \{p_a\}) = exp(-\alpha_a(\vec r - \vec{R}_a)^2)(x-X_a)^{p_{ax}}(y-Y_a)^{p_{ay}}(Z-z_a)^{p_{az}}.
$$

Можно заметить, что такая функция является произведением трех функций одной переменной:
$$
G(\vec{r}; \vec{R_a}, \alpha_a, \{p_a\}) = G(x; X_a, \alpha_a, p_{ax})G(y; Y_a, \alpha_a, p_{ay})G(z; Z_a, \alpha_a, p_{az}),
$$

$$
G(x; X_a, \alpha_a, p_{ax}) = exp(-\alpha_a(x - X_a)^2)(x-x_a)^{p_{ax}}.
$$

Таким образом мы могли бы свести наш интеграл к произведению трех одномерных интегралов, если бы не множитель
$\frac{1}{|\vec{r}-\vec{r'}|}$.

Воспользуемся следующим соотношением, правильность которого может проверить читатель:
$$
\frac{1}{|\vec{r}-\vec{r'}|} = \frac{1}{2\pi^2}\int d^3\vec{q}\frac{1}{q^2}exp(i\vec{q}(\vec{r}-\vec{r'})),
$$
$$ 
\frac{1}{|\vec{r}-\vec{r'}|} = \frac{1}{2\pi^2}\int_0^{\infty} d\lambda \int d^3\vec{q}exp(i\vec{q}(\vec{r}-\vec{r'}) -\lambda q^2).
$$

Теперь наш интеграл можно свести к интегралу от произведения трех одномерных интегралов.

## Действие оператора сдвига и более общих линейных функционалов на одномерные гауссианы
Для дальнейшего упрощения рассматриваемого интеграла заметим, что произведение двух гауссиан также будет
гауссианой:
$$
exp(-\alpha_a(x-X_a)^2 + -\alpha_b(x-X_b)^2) = exp(-\alpha'(x-X')^2)exp(-\alpha_aX_a^2-\alpha_b X_b^2+\alpha'X'^2),
$$
новое положение максимума и коэффициент в показателе экспоненты определены как:
$$
\alpha' = \alpha_a+\alpha_b,\\
X' = \frac{\alpha_a X_a + \alpha_b X_b}{\alpha_a + \alpha_b}.
$$

В нашем случае, кроме гауссианы надо преобразовать еще полином перед экспонентой --
$(x-X_a)^{p_{ax}}(x-X_b)^{p_{bx}}$.
Вместо явного выписывания коэффициентов при (x-X') для этого частного случая рассмотрим более общую задачу переразложения произвольного полинома P(x) на новый центр:
$$
P\tilde(x-X') = T(X'-X_a)P(x-X_a) = P(x-X' + (X'-X_a)),\\
P\tilde(x-X') = \sum C^n_k(X'-X_a)^{n-k}(x-X')^k.
$$
Таким образом можно получить новые полиномы, центрированные на $X'$, а не на $X_a$ или $X_b$,
интересующий нас полином будет являться их произведением.

Задача переразложения полинома на новый центр является частным случаем следующей задачи (СВЕРТКА) - пусть есть некоторый полином $P(x)$ и функция $f(x)$, нужно найти полином P'(y), который удовлетворяет следующему соотношению:
$$
P'(y) = S(k, f)P(x)=\int_{-\infty}^{\infty} dx P(x+ky) f(x).
$$

Пусть $P'(y) = \sum {c'}_n y^n $, $f_n = \int dx x^n f(x)$, тогда

$$
c'_n = \sum_{m} C^m_n k^n f_{m-n}c_m.
$$

Сдвиг на константу $X' - X_a$ соответствует выбору $f(x) = \delta(x-(X'-X_a))$, $k=1$. 

Реализация указанных операций в коде зависит от выбора конкретного представления для полиномов.
Для наших задач удобно воспользоваться представлением в виде столбца коэффициентов при каждой степени:
$$P(x) =(1\ x\ \cdots x^n)\left(
\begin{array}{c} c_0 \\ c_1 \\ \cdots \\ c_n \end{array} 
\right).$$ 

Ниже приведен код, написанный для такого представления полиномов, функций **poly_mul**, возращающей произведение двух полиномов и
**poly_func**, возвращающей результат действия оператора $S(k, f)$. 

Функции **trans** для сдвига полинома в новый центр и **poly_gaus_int**, возвращающей результат свертки с гаусианой используют вызов  функции **poly_func**.

In [ ]:
def poly_mul(p1s, p2s):
    res = np.zeros(len(p1s)+len(p2s), dtype=complex)
    inds = np.arange(len(p1s))
    m1 = inds[abs(p1s)>1e-15]
    m2 = np.arange(len(p2s))[abs(p2s)>1e-15]
    for ii in m1:
        for jj in m2:
            res[ii+jj] += p1s[ii]*p2s[jj]
    return res

def poly_func(p1s, k, fs):
    res = np.zeros_like(p1s, dtype=complex)
    inds = np.arange(len(p1s))
    inds = inds[abs(p1s)>1e-15]
    for ii in inds:
        js = np.arange(ii+1)
        res[js] += k**js*fs[ii-js]*sp.binom(ii, js)*p1s[ii]
    return res

def trans(p, dx):
    return poly_func(p, 1e0, dx**np.arange(_lmax+1))

def poly_gaus_int(a, p, k):
    t = _gaus_int*a**(-5e-1*(np.arange(_lmax+1)+1))
    return poly_func(p, k, t)

## Преобразование 4-х центрового интеграла в двух центровой
Теперь у нас все готово для того, чтобы провести первое упрощение интеграла
I(ABCD), для сокращения записи будем использовать нотацию вида $T(\vec{R})P(\vec{r})$, которая
означает, что операция сдвига проводится для каждой из компонент x, y, z. Будем хранить
коэффициенты многочленов для x, y, z в виде матрицы 3 x $l_{max}$ в коде.

Преобразуем произведение $ G_1 =  G(\vec r;\vec{R}_a, \alpha_a,\{p_a\}) G(\vec r; \vec{R}_b, \alpha_b,\{p_b\})$:
$$
G_1 = exp(-\alpha_a R_a^2 - \alpha_b R_b^2 + \alpha'R_1^2)\left(T(\vec{R_1}-\vec{R_A})P_A(\vec{r}-\vec{R_A})\right)\cdot\\
\left(T(\vec{R_1}-\vec{R_B})P_B(\vec{r}-\vec{R_A})\right).
$$

Таким образом $G_1$ есть произведение гауссианы и полинома, центрированных в $\vec{R}_1$.
Аналогичным образом преобразуем произведение $ G_2 =  G(\vec r';\vec{R}_c, \alpha_c,\{p_c\}) G(\vec r'; \vec{R}_d, \alpha_d,\{p_d\})$.
Ниже приведен код функции **gaus_mul**, которая принимает параметры двух гауссиан и полиномов и
возвращает параметры их произведения:

In [ ]:
def gaus_mul(R1, a1, p1s, R2, a2, p2s):
    R = a1*R1+a2*R2
    a = a1+ a2
    R /= a
    dr1, dr2 = R - R1, R - R2
    com_mul = np.exp(np.sum(-a1*R1**2-a2*R2**2+a*R**2)/3)
    ps = []
    for ii in range(3):
        p = poly_mul(trans(p1s[ii], dr1[ii]), trans(p2s[ii], dr2[ii]))
        p *= com_mul
        ps +=[p]
    return R, a, ps

## Интегрирование по пространственным координатам
После проведенных выше преобразований интеграл $I(ABCD)$ принимает вид:
$$
I(ABCD) =  \frac{1}{2\pi^2}\int_0^{\infty} d\lambda \int d^3\vec{q}exp(i\vec{q}(\vec{r}-\vec{r'}) -\lambda q^2) G_1(\vec{r})G_2(\vec{r'}),  
$$
$$
I(ABCD) = \frac{1}{2\pi^2}\int_0^{\infty} d\lambda \int d^3\vec{q}e^{-\lambda q^2}\left(\int dV e^{i\vec{q}\vec{r}}G_1(\vec{r})\right)\left(\int dV' e^{-i\vec{q}\vec{r'}}G_2(\vec{r'})\right).
$$

Интеграл по dV и аналогичный ему интеграл по dV' может быть представлен в виде действия операции $S$, введенной ранее:
$$
\int dV e^{i\vec{q}\vec{r}}G_1(\vec{r}) = \int dV exp\left(i\vec{q}\vec{r}-\alpha_1(\vec{r}-\vec{R_1})^2\right)P_1(\vec{r}-\vec{R_1}),
$$

$$
\int dV e^{i\vec{q}\vec{r}}G_1(\vec{r}) = exp\left(-q^2/(4\alpha_1)+i\vec{q}\vec{R_1}\right)(S(\frac{1}{2i}, f) P_1(\vec{r}-\vec{R_1})) =  exp\left(-q^2/(4\alpha_1)+i\vec{q}\vec{R_1}\right)\tilde P_1(q).
$$
Функция $f$ здесь $f(\vec r) = e^{-\alpha_1 \vec r^2}$. Интегрирование по dV' проводится аналогично.

## Интегрирование по q
Окончательный вид $I(ABCD)$:
$$
I(ABCD) = \int_{0}^{\infty}\int d^3\vec{q} \tilde P_1(\vec{q}) \tilde P_2(\vec{q})exp(-q^2(\frac{1}{4\alpha_1} + \frac{1}{4\alpha_2} + \lambda) + i\vec{q}(\vec{R_1}-\vec{R_2})).
$$

Пусть $\sigma = \left(\frac{1}{4\alpha_1} + \frac{1}{4\alpha_2} + \lambda\right)^{-1}$, $d\lambda = -d\sigma/\sigma^2$, $\sigma_0 = \left(\frac{1}{4\alpha_1} + \frac{1}{4\alpha_2} \right)^{-1}$:

$$
I(ABCD) = \int_0^{\sigma_0}\frac{d\sigma}{\sigma^2}exp(-\frac{\sigma}{4}(\vec{R}_1-\vec{R}_2)^2)S(\frac{i \sigma}{2}, exp^{-1/\sigma \vec{q}})(\tilde P_1(\vec{q}) \tilde P_2(\vec{q})).
$$

## Интегрирование по sigma(lambda)
Операция S в последнем выражении не может быть выполнена с помощью **poly_func**, поскольку k и f зависят  от $\sigma$, по которой будет идти интегрирование. Поэтому все коэффициенты должны быть посчитаны вручную, при проведении вычислений следует учитывать, что каждая степень k дает такую же степень $sigma$,
а интегрирование $q^n exp(...)$ по q дает дополнительную cтепень (n+1)/2).

После проведения всех преобразований получаем, что:
$$
I(ABCD) = \int_0^{\sigma_0} d\sigma \sigma^{1/2} P''(\sigma)exp(-\frac{\sigma}{4}(\vec{R_1}-\vec{R_2})^2).
$$
Последний интеграл берется в элементарных функциях если $\vec{R_1}=\vec{R_2}$ и выражается через 
неполные гамма функции в противном случае.
Ниже приведен код функций **sigma_int** и **sigma_int0** для его вычисления. Код для вычисления
коэффициентов при степенях $\sigma$ представлен в функции **sigma_poly**.
Код функции **twoint_coul** осуществляющий полное вычисление I(ABCD)  также приведен ниже.

In [ ]:
def sigma_int(rr, sigma0, psigma):
    assert(rr>=0)
    if (rr<1e-15):
        return sigma_int0(sigma0, psigma)
    res = 0e0
    if abs(sigma0*rr)<1e-2:
        aux_poly = np.zeros_like(psigma)
        mm = np.arange(10)
        aux_poly[mm] = rr**mm/sp.factorial(mm)*(-1)**mm
        return sigma_int0(sigma0, poly_mul(aux_poly, psigma))
    for ii, c in enumerate(psigma):
        if abs(c)<1e-15:
            continue
        k = ii+0.5
        res += c*sp.gamma(k)*sp.gammainc(k,sigma0*rr)/rr**k   
    return res

def sigma_int0(sigma0, psigma):
    res = 0e0
    for ii, c in enumerate(psigma):
        if abs(c)<1e-15:
            continue
        k = ii+0.5
        res += sigma0**k/k*c   
    return res
def sigma_poly(dx, p):
    res = np.zeros(_ps_max, dtype=complex)
    for ii, cp  in enumerate(p):
        if abs(cp)<1e-15:
            continue
        for jj in range(ii+1):
            kk = ii-jj
            if kk % 2 != 0:
                continue
            res[int(kk/2)+jj] += (sp.gamma(0.5*(kk+1))*sp.binom(ii, jj)*dx**jj/2**jj*
                                      (1j)**jj*cp)
    return res
    
def twoint_coul(ga, gb, gc, gd):
    ra, aa, pa = ga
    rb, ab, pb = gb
    rc, ac, pc = gc
    rd, ad, pd = gd
    #4 center to 2 center
    r1, a1, p1 = gaus_mul(ra, aa, pa, rb, ab, pb)
    r2, a2, p2 = gaus_mul(rc, ac, pc, rd, ad, pd)
    #print(r1, a1, p1)
    #print(r2, a2, p2)
    #from r to q
    pq = []
    for ii in range(3):
        pq1 = poly_gaus_int(a1, p1[ii], 0.5j/a1)
        pq2 = poly_gaus_int(a2, p2[ii], -0.5j/a2)
        pq += [poly_mul(pq1, pq2)]
    #treat pq 
    #print(pq)
    #print('---'*10)
    sigma_fin = sigma_poly(r1[0]-r2[0], pq[0])
    sigma_fin = poly_mul(sigma_fin, sigma_poly(r1[1]-r2[1], pq[1]))
    sigma_fin = poly_mul(sigma_fin, sigma_poly(r1[2]-r2[2], pq[2]))
    #print(sigma_pols)
    return np.real(1/(2*np.pi**2)*sigma_int(np.dot(r1-r2,r1-r2)/4e0, 4e0/(1/a1+1/a2), sigma_fin))

In [ ]:
def primp(ms):
    res = [np.zeros(m+1, dtype=complex) for m in ms]
    for p in res:
        p[-1] = 1e-0
    return res

pt = np.zeros(_lmax+1)
pt[2] = 1
ptt = trans(pt, 1e0)
ptt3 = trans(ptt, -1e0)
#print(pt, ptt3)
#poly_gaus_int(0.3, ptt, 1e0),np.pi**0.5*(1/0.3**0.5+1/0.3**1.5*0.5), np.pi**0.5*0.5/0.3**1.5
r0 = np.zeros(3)
ra, rb, rc, rd = np.zeros((4, 3))
ra[2]=0.5
rb[2]=0.5
rc[2]=-0.5
rd[2]=-0.5
aa, ab, ac, ad = np.ones(4)
pa = primp([0,0,0])
pb = primp([0,0,0])
pc = primp([0,0,0])
pd = primp([0,0,0])
twoint_coul((ra,aa,pa), (rb,ab, pb), (rc, ac, pc), (rd, ad, pd))

# Тесты 
Прежде чем проводить проверку функции **twoint_coul** как целого, проведем тестирование всех функций, которые она вызывает:
- **poly_gaus_int**
- **gaus_mul**
- **poly_mul**
- **trans**;


Для этого напишем функции, которые вычисляют одноэлектронные интегралы через введенные ранее операции.
Рассмотрим интеграл перекрывания:
$$
S(AB) = \int dV G(r;R_a,\alpha_a, \{p_a\}) G(r;R_b, \alpha_b, \{p_b\}).
$$

$S(AB)$ может быть вычислен двумя способами -- непосредственное интегрирование по обьему с _предварительным_ приведением к одному центру и интеграл через преобразование Фурье:

$$
S(AB) = \frac{1}{(2\pi)^3}\int d^3\vec{q} dV dV' G(r;R_a,\alpha_a, \{p_a\})G(r';R_b, \alpha_b, \{p_b\})e^{i\vec{q}(\vec{r}-\vec{r'})}.
$$

Первый способ требует применения операции $T$ и может быть осуществлен с помощью функции **gaus_mul**
Второй способ требует применения двух операций $S$. Код для вычисления интеграла этими способами представлен в функциях **sint** и **sint2** соответственно, функция **gint** осуществляет одномерных функций $exp(-\alpha_a x^2)P_a(x)$  по переменной $x$.

In [ ]:
def gint(aa, pa):
    t = _gaus_int[:len(pa)]*aa**(-5e-1*(np.arange(len(pa))+1))
#    print(f"gint {t}")
    return np.sum(pa*t)

def sint(ga, gb):
    ra, aa, pa = ga
    rb, ab, pb = gb  
    r, a, p = gaus_mul(ra, aa, pa, rb, ab, pb)

    return np.real_if_close(gint(a, p[0])*gint(a, p[1])*gint(a, p[2]))

def sint2(ga, gb):
    ra, aa, pa = ga
    rb, ab, pb = gb
    qa = 1e0/4*(1/aa + 1/ab)
#    print(f"sint2 {qa}")
    res = 1e0
    t = []
    for ii in range(3):
        t += [(ra[ii]-rb[ii])**np.arange(len(pa[ii])+len(pb[ii]))]
    for ii in range(3):
        pqa = poly_gaus_int(aa, pa[ii], 0.5j/aa)
        pqb = poly_gaus_int(ab, pb[ii], -0.5j/ab)
        pq = poly_mul(pqa, pqb)
        pq = poly_gaus_int(qa, pq, 0.5j/qa)*np.exp(-(ra[ii]-rb[ii])**2/(4*qa))
        res = res*np.dot(pq, t[ii])
    return np.real_if_close(res)/(2*np.pi)**3

In [ ]:
ga = (np.zeros(3), 1e0, primp([0, 0, 0]))
gb = (np.array([0e0,0e0, 1e0]), 1e0, primp([0, 0, 2]))
gc = (np.array([1e0,1e0, 0e0]), 1e0, primp([0, 1, 2]))
print(sint(ga, ga)/(0.5**1.5*np.pi**1.5), sint2(ga, ga)/(0.5**1.5*np.pi**1.5))
print(sint(ga, gb)/(0.5**1.5*np.pi**1.5), sint2(ga, gb)/(0.5**1.5*np.pi**1.5))
print(sint(gb, gb)/(0.5**1.5*np.pi**1.5), sint2(gb, gb)/(0.5**1.5*np.pi**1.5))
print(sint(gb, gc)/(0.5**1.5*np.pi**1.5), sint2(gb, gc)/(0.5**1.5*np.pi**1.5))
print(gaus_mul(*gb, *gc))

Аналогично можно рассмотреть интегралы, возникающие при вычислении 
кинетической энергии:
$$
T(AB) = \int dV G(\vec r;\vec R_a, \alpha_a, \{p_a\}) \Delta G(\vec r; \vec R_b,\alpha_b, \{p_b\}).
$$

Здесь перед приведением к одному центру провести дифференцирование по пространственным координатам.
Код для полиномиальной части производной от функций одной переменной вида $exp(-\alpha x^2)P(x)$ представлен в **dxgaus**. Код для вычисления $T(AB)$ первым способом представлен в **tint**.
Код для вычисления $T(AB)$ через преобразование Фурье представлен в **tint2**.


In [ ]:
def dxgaus(a, p):
    res = np.zeros(len(p)+1, dtype=complex)
    nn = np.arange(len(p))
    if len(p)>1:
        res[:-2] += nn[1:]*p[1:]
    res[1:] -= 2*a*p
    return res

def tint_ii(ii, ga, gb):
    ra, aa, pa = ga
    rb, ab, pb = gb
    new_pb = pb.copy()
    new_pb[ii] = dxgaus(ab, dxgaus(ab, pb[ii]))
    r, a, p = gaus_mul(ra, aa, pa, rb, ab, new_pb)
#    print(f" tint_ii {pa}")
#    print(f" tint_ii {new_pb}")
    return np.real_if_close(gint(a, p[0])*gint(a, p[1])*gint(a, p[2]))

def tint2_ii(mm, ga, gb):
    ra, aa, pa = ga
    rb, ab, pb = gb
    qa = 1e0/4*(1/aa + 1/ab)
#    print(f"tint2 {qa}")
    res = 1e0
    for ii in range(3):
        pqa = poly_gaus_int(aa, pa[ii], 0.5j/aa)
        pqb = poly_gaus_int(ab, pb[ii], -0.5j/ab)
        pq = poly_mul(pqa, pqb)
        if ii == mm:
            tmp = np.zeros(len(pq)+2, dtype =complex)
            tmp[2:] = -pq[:]
            pq = tmp
 #           print(f"tint2_ii new_pq {pq}") 
 #           print(f"tint2_ii t {t[ii]}") 
        pq = poly_gaus_int(qa, pq, 0.5j/qa)*np.exp(-(ra[ii]-rb[ii])**2/(4*qa))
        t = (ra[ii]-rb[ii])**np.arange(len(pq))
        res = res*np.dot(pq, t)
    return np.real_if_close(res)/(2*np.pi)**3


def tint(ga, gb):
    return sum(tint_ii(ii, ga, gb) for ii in range(3))

def tint2(ga, gb):
    return sum(tint2_ii(ii, ga, gb) for ii in range(3))



In [ ]:
print(tint(ga, ga), tint2(ga, ga))
print(tint(ga, gb), tint2(ga, gb))
print(tint(gb, ga), tint2(gb, ga))
print(tint(gb, gb), tint2(gb, gb))
print(tint(ga, gc), tint2(ga, gc))
print(tint(gc, gc), tint2(gc, gc))

Оставшийся одноэлектронный интеграл $U(AB)$ можно взять
только одним способом:

$$
U(AB) =  \int dV G(\vec r;\vec R_a, \alpha_a, \{p_a\})  G(\vec r; \vec R_b,\alpha_b, \{p_b\})\frac{1}{|\vec r - \vec R_C|}.
$$

In [ ]:
def uint(ga, gb, rc):
    ra, aa, pa = ga
    rb, ab, pb = gb
    #4 center to 2 center
    r1, a1, p1 = gaus_mul(ra, aa, pa, rb, ab, pb)
    r2 = rc
    #from r to q
    pq = np.zeros_like(p1, dtype=complex)
    for ii in range(3):
        pq[ii, :] = poly_gaus_int(a1, p1[ii, :], 0.5j/a1)
    #treat pq 
    sigma_fin = sigma_poly(r1[0]-r2[0], pq[0])
    sigma_fin = poly_mul(sigma_fin, sigma_poly(r1[1]-r2[1], pq[1]))
    sigma_fin = poly_mul(sigma_fin, sigma_poly(r1[2]-r2[2], pq[2]))
    #print(sigma_pols)
    return 1/(2*np.pi**2)*sigma_int(np.dot(r1-r2,r1-r2)/4e0, 4e0/(1/a1+1/a2), sigma_fin)

# Задачи

1. Приведенный здесь код работает примерно на два порядка медленней кодов используемых 
    в реальных квантовохимических пакетах. Причиной этого является использование не слишком эффективного     алгоритма, а также использование numpy для проведения всех операций. Единственными достоинствами этого метода вычисления двухэлектронных интегралов является простота его реализации и отладки.
     - Ускорить работу **twoint_coul** переписав наиболее часто используемые функции на fortran (интеграция с python с помощью _f2py_) или cython. Для поиска этих функций воспользоваться встроенным в ipython профайлером (команда %prun). Хорошим результатом считается ускорение в 6 и более раз.
     - Реализовать более эффективные алгоритмы для вычисления двух электронных интегралов и добиться их согласия с кодом, приведенным в этом тексте.

2. Воспользоваться функциями **sint, tint, twoint_coul** для оценки разницы энергий молекулы водорода в синглетном и триплетном состояниях. Использовать минимальный базис (1s гауссиана на каждом атоме), показатель экспоненты считать варьируемым параметром для достижения минимума энергии в основном состоянии. Межатомное расстояние  проварьировать для обоих состояний. Сравнить результаты с экспериментальными данными и расчетами реально использующимися методами.

# Тесты для двухэлектронных интегралов
Ниже проведено сравнение тестового набора (файл eri.dat скачивается отдельно) двухэлектронных интегралов для sto-3g базиса молекулы воды c результатами работы **twoint_coul**. Поскольку чтение базиса из текстового файла с использзованием регулярных выражений не является темой этой главы коментариев приводится не будет. Для проведения тестов нужно выполнить все ячейки снизу. Параметр \_eps определяет степень допустимости отличий от тестовых данных.


In [ ]:
import re
basis_buf ="""
*** Started AO integrals evaluation

basis set:
   Z                x          y          z        n  l  m   exponents
Z= 8  origin=   0.000000   0.000000   0.221664    (0, 0, 0) (130.70932,)   norm factors= [ 1.] 1
Z= 8  origin=   0.000000   0.000000   0.221664    (0, 0, 0) (23.808861,)   norm factors= [ 1.] 2
Z= 8  origin=   0.000000   0.000000   0.221664    (0, 0, 0) (6.4436083,)   norm factors= [ 1.] 3
Z= 8  origin=   0.000000   0.000000   0.221664    (0, 0, 0) (5.0331513,)   norm factors= [ 1.] 4
Z= 8  origin=   0.000000   0.000000   0.221664    (0, 0, 0) (1.1695961,)   norm factors= [ 1.] 5
Z= 8  origin=   0.000000   0.000000   0.221664    (0, 0, 0) (0.380389,)   norm factors= [ 1.]  6
Z= 8  origin=   0.000000   0.000000   0.221664    (0, 0, 1) (5.0331513,)   norm factors= [ 1.] 7
Z= 8  origin=   0.000000   0.000000   0.221664    (0, 1, 0) (5.0331513,)   norm factors= [ 1.] 8
Z= 8  origin=   0.000000   0.000000   0.221664    (1, 0, 0) (5.0331513,)   norm factors= [ 1.] 9
Z= 8  origin=   0.000000   0.000000   0.221664    (0, 0, 1) (1.1695961,)   norm factors= [ 1.] 10
Z= 8  origin=   0.000000   0.000000   0.221664    (0, 1, 0) (1.1695961,)   norm factors= [ 1.] 11
Z= 8  origin=   0.000000   0.000000   0.221664    (1, 0, 0) (1.1695961,)   norm factors= [ 1.] 12
Z= 8  origin=   0.000000   0.000000   0.221664    (0, 0, 1) (0.380389,)   norm factors= [ 1.]  13
Z= 8  origin=   0.000000   0.000000   0.221664    (0, 1, 0) (0.380389,)   norm factors= [ 1.]  14
Z= 8  origin=   0.000000   0.000000   0.221664    (1, 0, 0) (0.380389,)   norm factors= [ 1.]  15
Z= 8  origin=   0.000000   0.000000   0.221664    (0, 0, 2) (1.0,)   norm factors= [ 1.]       16
Z= 8  origin=   0.000000   0.000000   0.221664    (0, 1, 1) (1.0,)   norm factors= [ 1.]       17
Z= 8  origin=   0.000000   0.000000   0.221664    (0, 2, 0) (1.0,)   norm factors= [ 1.]       18
Z= 8  origin=   0.000000   0.000000   0.221664    (1, 0, 1) (1.0,)   norm factors= [ 1.]       19
Z= 8  origin=   0.000000   0.000000   0.221664    (1, 1, 0) (1.0,)   norm factors= [ 1.]       20
Z= 8  origin=   0.000000   0.000000   0.221664    (2, 0, 0) (1.0,)   norm factors= [ 1.]       21
Z= 1  origin=   0.000000   1.430893  -0.886655    (0, 0, 0) (3.42525091,)   norm factors= [ 1.] 22
Z= 1  origin=   0.000000   1.430893  -0.886655    (0, 0, 0) (0.62391373,)   norm factors= [ 1.] 23
Z= 1  origin=   0.000000   1.430893  -0.886655    (0, 0, 0) (0.1688554,)   norm factors= [ 1.]  24
Z= 1  origin=   0.000000  -1.430893  -0.886655    (0, 0, 0) (3.42525091,)   norm factors= [ 1.] 25
Z= 1  origin=   0.000000  -1.430893  -0.886655    (0, 0, 0) (0.62391373,)   norm factors= [ 1.] 26
Z= 1  origin=   0.000000  -1.430893  -0.886655    (0, 0, 0) (0.1688554,)   norm factors= [ 1.]  27"""
gpat = re.compile("Z=\s+\d+\s+origin=\s+([+-]?\d+\.\d+)\
\s+([+-]?\d+\.\d+)\s+([+-]?\d+\.\d+)\s+\((\d+),\s+(\d+),\s+(\d+)\) \(([+-]?\d+\.\d+),\s*\)")
basis = []
for line in basis_buf.split("\n"):
    if gpat.match(line) is not None:
        #print(line)
        #print(gpat.match(line).groups())
        x, y, z, mx, my, mz, alpha = map(float, gpat.match(line).groups())
        mx, my, mz = map(int, [mx, my, mz])
        basis += [(np.array([x, y, z]), alpha,primp([mx, my, mz]))]
        #print(basis[-1])


In [ ]:
_eps = 1e-9
def make_twoints(basis):
    resval, resinds = [], []
    ci = 0
    for ia in range(len(basis)):
        ga = basis[ia]                
        print(ci)
        for ib in range(ia,len(basis)):
            gb = basis[ib]
            for ic in range(ia, len(basis)):
                gc = basis[ic]
                q0 = ib if ic == ia else ic
                for iid in range(q0, len(basis)):
                    gd = basis[iid]
                    tmp = twoint_coul(ga, gb, gc, gd)
                    if abs(tmp)<_eps:
                        continue
                    resval += [tmp]
                    resinds += [[ia+1, ib+1, ic+1, iid+1]]
        ci +=1
    return resval, resinds
vals, v_inds = make_twoints(basis)

In [ ]:
tst_data, tst_inds = [], []
for line in open("eri.dat"):
    aux=line.split()
    if abs(float(aux[-1]))<_eps:
        continue
    tst_data += [float(aux[-1])]
    tst_inds += [list(map(int, aux[:-1]))]
ci = 0
cerr=0
maxerr =0e0
meanerr = 0e0
for v, v_ind, t, t_ind in zip(vals, v_inds, tst_data, tst_inds):
    if ci % 977 == 0:
        print("all ok here ", t_ind, np.real(v), t)
    if any(vi != ti for vi,ti in zip(v_ind, t_ind)):
        raise ValueError(f"something wrong with inds {v_ind} {t_ind}")
    if abs(v-t)>_eps:
        cerr +=1
        maxerr = max(maxerr, abs(v-t)/abs(v))
        meanerr += abs(v-t)/abs(v)
        if maxerr == abs(v-t)/abs(v):
            print(t_ind, np.real(v), t, abs(v-t), maxerr)
    if v*t<0:
        print("WRONG SIGN?")
        print(t_ind, np.real(v), t, abs(v-t), maxerr)
    ci +=1
print(cerr, ci)
print(f"maxerr:{maxerr}, meanerr:{meanerr/cerr}")